## Standard Imports

In [2]:
import ee
import os
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO


print('Imported')

Imported


In [3]:

# Initialize Earth Engine
try:
    ee.Initialize(project = 'glacier-probe-model-475519')
except:
    ee.Authenticate()
    ee.Initialize(project = 'glacier-probe-model-475519')

print('Done')

Done


## 1. Hasdeo Forest RGB Downloader

In [3]:
"""
Hasdeo Forest JPEG Downloader — 1000+ IMAGE DATASET
=====================================================
Exports RGB Sentinel-2 composites to Google Drive for the Hasdeo Arand
forest region. Files sync automatically to your local PC via Google Drive
desktop app (no manual download needed).

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMAGE COUNT BREAKDOWN
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Monthly composites   : 120 months × 7 regions  =  840
  Seasonal dry         :  10 years  × 7 regions  =   70
  Seasonal wet         :  10 years  × 7 regions  =   70
  ─────────────────────────────────────────────────────
  TOTAL POTENTIAL                                = 980+
  (actual count depends on cloud availability per period)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TIMELINE & PHASES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  pre_event    → 2015-01 – 2017-12  (baseline / pre-deforestation)
  first_event  → 2018-01 – 2023-12  (broad clearance window)
  second_event → 2022-03 – 2022-04  (peak clearance, nested tag)
  post_event   → 2024-01 – 2024-12  (recovery monitoring)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
OUTPUT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Drive folder  : Hasdeo_1000_Dataset/
  Format        : RGB GeoTIFF (B4/B3/B2, 30 m) — opens as colour image
  File size     : ~30–60 MB per image at 30 m resolution
  Naming        : MON_<Region>_<YYYY>_<MM>_<phase>.tif
                  SEAS_<DRY|WET>_<Region>_<YYYY>_<phase>.tif
  Local sync    : Install Google Drive desktop app → files appear in
                  your local "Google Drive" folder automatically.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LOCAL DATASET STRUCTURE (after Drive sync)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Google Drive/Hasdeo_1000_Dataset/   ← auto-synced to your PC
  ├── MON_Hasdeo_Full_2015_01_pre_event.tif
  ├── MON_Hasdeo_Full_2022_03_second_event.tif
  ├── SEAS_DRY_Hasdeo_Core_2021_first_event.tif
  └── ... (980+ files)

  Locally generated metadata (run location):
  hasdeo_1000_local/
  ├── metadata/
  │   ├── export_log.csv    ← per-image record (filename, phase, dates, task_id)
  │   └── manifest.json     ← full dataset summary
  └── logs/
      └── task_ids.json     ← all GEE task IDs for monitoring

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
SETUP & USAGE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  1. pip install earthengine-api tqdm
  2. earthengine authenticate        # one-time browser login
  3. python hasdeo_1000_downloader.py
  4. Monitor tasks: https://code.earthengine.google.com/tasks
  5. Files appear in Google Drive → syncs locally via desktop app
"""

import ee
import csv
import json
import time
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID        = 'glacier-probe-model-475519'  # ← your GEE Cloud project ID
OUTPUT_DIR        = 'hasdeo_1000_local'           # local metadata/logs directory
MAX_CLOUD_COVER   = 20                            # % cloud cover filter per scene
MIN_IMAGES_PERIOD = 1                             # skip period if fewer scenes
SCALE             = 30                            # 30 m → ~30–60 MB per image

# ── Single flat Drive folder — ALL images land here ──────────────────────
# Google Drive desktop app syncs this folder to your local PC automatically.
DRIVE_FOLDER = 'Hasdeo_1000_Dataset'

# ── 7 sub-regions covering Hasdeo Arand (WGS84: [west, south, east, north])
REGIONS = {
    'Hasdeo_Full':    [82.58, 22.40, 83.20, 22.90],  # entire block
    'Hasdeo_North':   [82.58, 22.65, 83.20, 22.90],  # northern half
    'Hasdeo_South':   [82.58, 22.40, 83.20, 22.65],  # southern half
    'Hasdeo_Core':    [82.75, 22.50, 83.05, 22.80],  # dense forest core
    'Hasdeo_East':    [82.95, 22.40, 83.20, 22.90],  # eastern edge
    'Hasdeo_West':    [82.58, 22.40, 82.90, 22.90],  # western edge
    'Hasdeo_Buffer':  [82.50, 22.30, 83.30, 23.00],  # extended buffer zone
}

# ── Bands selected for cloud masking (RGB export only needs B2/B3/B4) ────
S2_BANDS = ['B2', 'B3', 'B4', 'SCL']

# ─────────────────────────────────────────────────────────────────────────────
# PHASE LABELLING
# ─────────────────────────────────────────────────────────────────────────────

def get_phase(year: int, month: int) -> str:
    """
    Assign a deforestation phase label to each month composite.
      pre_event    → 2015–2017  (baseline forest cover)
      second_event → Mar–Apr 2022  (peak clearance event — nested within first)
      first_event  → 2018–2023  (broad deforestation window)
      post_event   → 2024       (recovery / monitoring phase)
    """
    if 2015 <= year <= 2017:
        return 'pre_event'
    if year == 2022 and month in (3, 4):
        return 'second_event'
    if 2018 <= year <= 2023:
        return 'first_event'
    if year == 2024:
        return 'post_event'
    return 'background'

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIOD BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    """
    Build monthly windows Jan 2015 – Dec 2024.
    10 years × 12 months = 120 periods per region.
    """
    periods = []
    for year in range(2015, 2025):          # 2015 → 2024 inclusive
        for month in range(1, 13):
            start = f'{year}-{month:02d}-01'
            end   = (f'{year + 1}-01-01'
                     if month == 12
                     else f'{year}-{month + 1:02d}-01')
            periods.append((start, end, get_phase(year, month), year, month))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# EARTH ENGINE INITIALISATION
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialised  |  project: {PROJECT_ID}")
    except Exception:
        print("⚠  Authenticating with Earth Engine (browser will open)...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialised.")

def create_local_dirs():
    """Create local directories for metadata and logs."""
    for sub in ['metadata', 'logs']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Local dirs ready: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    """
    Dual-layer cloud mask:
      QA60  — opaque cloud (bit 10) + cirrus (bit 11)
      SCL   — shadow (3), med-cloud (8), high-cloud (9),
               cirrus (10), snow/ice (11)
    """
    qa       = image.select('QA60')
    qa_mask  = (qa.bitwiseAnd(1 << 10).eq(0)
                  .And(qa.bitwiseAnd(1 << 11).eq(0)))
    scl      = image.select('SCL')
    scl_mask = (scl.neq(3) .And(scl.neq(8))
                            .And(scl.neq(9))
                            .And(scl.neq(10))
                            .And(scl.neq(11)))
    return image.updateMask(qa_mask.And(scl_mask))


def get_s2_collection(roi, start_date, end_date):
    """
    Return a cloud-masked Sentinel-2 SR collection.
    Selects only bands needed (B2/B3/B4 for RGB + SCL + QA60 for masking).
    Keeping the band selection minimal speeds up GEE processing.
    """
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['QA60'])
              .map(mask_s2_clouds))

# ─────────────────────────────────────────────────────────────────────────────
# RGB EXPORT TO DRIVE
# ─────────────────────────────────────────────────────────────────────────────

def export_rgb_to_drive(composite, description, roi):
    """
    Visualise composite as RGB (B4/B3/B2) and export to Drive.

    Why GeoTIFF instead of JPEG?
    GEE's toDrive() only supports GeoTIFF and TFRecord formats — native
    JPEG export is not available via the batch API. The resulting file is
    a 3-band (Uint8) RGB GeoTIFF that opens identically to a JPEG in
    any image viewer, QGIS, Photoshop, or Python (PIL/rasterio).

    At 30 m resolution, expected file size per image: ~30–60 MB.

    To convert to true .jpg after Drive sync (requires GDAL):
        gdal_translate -of JPEG -co QUALITY=90 input.tif output.jpg
    """
    rgb = composite.visualize(
        bands=['B4', 'B3', 'B2'],
        min=200,
        max=3000,
        gamma=1.4,
    )
    task = ee.batch.Export.image.toDrive(
        image          = rgb,
        description    = description,    # task label in GEE console
        folder         = DRIVE_FOLDER,   # flat Drive folder (no nesting)
        fileNamePrefix = description,    # filename = description + .tif
        region         = roi,
        scale          = SCALE,          # 30 m
        crs            = 'EPSG:32644',   # UTM Zone 44N — Hasdeo, Chhattisgarh
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True},
    )
    task.start()
    return task

# ─────────────────────────────────────────────────────────────────────────────
# SEASONAL COMPOSITE BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def make_seasonal_composite(roi, year, season):
    """
    Median composite for a season window:
      dry → Nov–Apr  (low cloud cover; optimal for deforestation detection)
      wet → May–Oct  (captures phenological/greenness changes)
    Returns (composite, scene_count, start_str, end_str).
    """
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'
    col   = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    return col.median().clip(roi), count, start, end

# ─────────────────────────────────────────────────────────────────────────────
# METADATA TRACKING
# ─────────────────────────────────────────────────────────────────────────────

records = []

def log_record(task_id, description, region,
               start, end, count, phase, ctype):
    """Append one export record to the in-memory list."""
    records.append({
        'filename':       f'{description}.tif',
        'drive_folder':   DRIVE_FOLDER,
        'description':    description,
        'region':         region,
        'composite_type': ctype,
        'phase':          phase,
        'start_date':     start,
        'end_date':       end,
        'scene_count':    count,
        'task_id':        task_id,
        'scale_m':        SCALE,
        'bands':          'B4,B3,B2 (RGB visualised, Uint8)',
        'crs':            'EPSG:32644',
        'approx_size_mb': '30–60',
        'exported_at':    datetime.now().isoformat(),
    })


def save_csv():
    if not records:
        return
    p = f'{OUTPUT_DIR}/metadata/export_log.csv'
    with open(p, 'w', newline='') as f:
        w = csv.DictWriter(f, fieldnames=records[0].keys())
        w.writeheader()
        w.writerows(records)
    print(f"✓ CSV     → {p}  ({len(records)} records)")


def save_manifest(all_tasks, exported, skipped, potential):
    """Write a full dataset summary JSON."""
    manifest = {
        'dataset':          'Hasdeo 1000 JPEG Dataset',
        'generated_at':     datetime.now().isoformat(),
        'drive_folder':     DRIVE_FOLDER,
        'local_metadata':   OUTPUT_DIR,
        'regions':          list(REGIONS.keys()),
        'timeline':         '2015-01-01 – 2024-12-31',
        'phases': {
            'pre_event':    '2015–2017  (baseline forest cover)',
            'first_event':  '2018–2023  (broad deforestation window)',
            'second_event': '2022-03 – 2022-04  (peak clearance event)',
            'post_event':   '2024       (recovery monitoring)',
        },
        'composite_types':  ['monthly', 'seasonal_dry', 'seasonal_wet'],
        'scale_m':          SCALE,
        'crs':              'EPSG:32644',
        'bands':            'B4, B3, B2  (RGB GeoTIFF, Uint8)',
        'approx_size_mb':   '30–60 per image',
        'summary': {
            'potential_images': potential,
            'tasks_queued':     exported,
            'skipped':          skipped,
            'estimated_gb':     f'{exported * 45 / 1024:.1f} GB  (at ~45 MB avg)',
        },
        'local_sync_note': (
            'Install Google Drive desktop app. Once GEE tasks complete, '
            f'files appear under Google Drive/{DRIVE_FOLDER}/ and sync '
            'automatically to your PC — no manual download required.'
        ),
        'monitor_url':     'https://code.earthengine.google.com/tasks',
        'tasks': [
            {'id': t.id, 'description': t.config.get('description', '')}
            for t in all_tasks if t
        ],
    }
    p = f'{OUTPUT_DIR}/metadata/manifest.json'
    with open(p, 'w') as f:
        json.dump(manifest, f, indent=2)
    print(f"✓ Manifest → {p}")


def save_task_ids(all_tasks):
    p = f'{OUTPUT_DIR}/logs/task_ids.json'
    with open(p, 'w') as f:
        json.dump(
            [{'id': t.id, 'description': t.config.get('description', '')}
             for t in all_tasks if t],
            f, indent=2,
        )
    print(f"✓ Task IDs → {p}")

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():

    # ── Counts ──────────────────────────────────────────────────────────────
    n_regions    = len(REGIONS)
    monthly_pot  = len(TIME_PERIODS) * n_regions   # 120 × 7 = 840
    seasonal_pot = 10 * n_regions * 2              # 10yr × 7 × 2 = 140
    total_pot    = monthly_pot + seasonal_pot       # 980

    print("=" * 70)
    print("  HASDEO FOREST — 1000 IMAGE JPEG DATASET DOWNLOADER")
    print("=" * 70)
    print(f"  Drive folder    : {DRIVE_FOLDER}/")
    print(f"  Local sync      : Google Drive desktop app → auto-sync to PC")
    print(f"  Resolution      : {SCALE} m  (~30–60 MB per image)")
    print(f"  Bands           : B4 / B3 / B2  (RGB, Uint8)")
    print(f"  Timeline        : Jan 2015 – Dec 2024  (10 years)")
    print(f"  Regions         : {n_regions}  → {list(REGIONS.keys())}")
    print(f"  Potential total : {total_pot} images")
    print(f"    ├─ Monthly    : {monthly_pot}   (120 months × {n_regions} regions)")
    print(f"    └─ Seasonal   : {seasonal_pot}   (10 yr × {n_regions} regions × 2 seasons)")
    print(f"  Est. local size : ~{total_pot * 45 // 1024} GB  (at 45 MB avg per image)")
    print("=" * 70)

    init_ee()
    create_local_dirs()

    all_tasks = []
    exported  = 0
    skipped   = 0

    # ════════════════════════════════════════════════════════════════════════
    # PART 1 — Monthly composites  (Jan 2015 – Dec 2024, 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n[ PART 1 ]  Monthly composites  (Jan 2015 – Dec 2024)")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for start, end, phase, year, month in tqdm(
                TIME_PERIODS,
                desc=f"  {region_name[:20]}",
                unit='month'):
            try:
                col   = get_s2_collection(roi, start, end)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PERIOD:
                    skipped += 1
                    continue

                composite = col.median().clip(roi)

                # e.g. MON_Hasdeo_Full_2022_03_second_event
                desc = f'MON_{region_name}_{year}_{month:02d}_{phase}'[:90]

                task = export_rgb_to_drive(composite, desc, roi)
                all_tasks.append(task)
                log_record(task.id, desc, region_name,
                           start, end, count, phase, 'monthly')
                exported += 1
                time.sleep(0.35)   # gentle GEE rate-limiting

            except Exception as e:
                print(f"\n    ✗ [{region_name} {start}]: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # PART 2 — Seasonal composites  (dry + wet, 2015–2024, 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n\n[ PART 2 ]  Seasonal composites  (dry Nov–Apr + wet May–Oct)")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for year in range(2015, 2025):          # 2015–2024
            for season in ('dry', 'wet'):
                try:
                    composite, count, s_date, e_date = make_seasonal_composite(
                        roi, year, season)

                    if count < MIN_IMAGES_PERIOD:
                        print(f"    ⊙ {season} {year}: no scenes — skipped")
                        skipped += 1
                        continue

                    # Determine phase for seasonal label
                    if year <= 2017:
                        phase = 'pre_event'
                    elif year == 2022 and season == 'dry':
                        phase = 'second_event'   # dry-2022 overlaps peak event
                    elif year <= 2023:
                        phase = 'first_event'
                    else:
                        phase = 'post_event'

                    # e.g. SEAS_DRY_Hasdeo_Core_2022_second_event
                    desc = f'SEAS_{season.upper()}_{region_name}_{year}_{phase}'[:90]

                    task = export_rgb_to_drive(composite, desc, roi)
                    all_tasks.append(task)
                    log_record(task.id, desc, region_name,
                               s_date, e_date, count, phase,
                               f'seasonal_{season}')
                    exported += 1
                    print(f"    ✓ {season.upper()} {year}: "
                          f"{count} scenes → {desc}")
                    time.sleep(0.35)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # SUMMARY
    # ════════════════════════════════════════════════════════════════════════
    print("\n" + "=" * 70)
    print("  EXPORT SUMMARY")
    print("=" * 70)
    print(f"  ✓ GEE tasks queued  : {exported} images")
    print(f"  ⊙ Skipped           : {skipped}  (insufficient cloud-free scenes)")
    print(f"  📦 Est. total size  : ~{exported * 45 // 1024} GB  (at ~45 MB avg)")
    print()
    print(f"  📁 Drive folder     : {DRIVE_FOLDER}/")
    print(f"  🔗 Monitor tasks    : https://code.earthengine.google.com/tasks")
    print()
    print("  HOW TO GET FILES ON YOUR LOCAL PC:")
    print("  ─────────────────────────────────────────────────────────────")
    print("  1. Wait for GEE tasks to complete (check monitor link above)")
    print("     Large exports: 10 min – several hours depending on region")
    print("  2. Open Google Drive in browser — files appear in:")
    print(f"     My Drive / {DRIVE_FOLDER}/")
    print("  3. Google Drive desktop app syncs them to your PC automatically")
    print("     Default local path (Windows): C:/Users/<you>/Google Drive/")
    print("                         (Mac)   : ~/Google Drive/")
    print()
    print("  OPTIONAL — convert RGB GeoTIFF → true JPEG (requires GDAL):")
    print("    Windows (PowerShell):")
    print("      Get-ChildItem *.tif | ForEach-Object {")
    print("        gdal_translate -of JPEG -co QUALITY=90 $_.Name ($_.BaseName+'.jpg')")
    print("      }")
    print("    Mac/Linux:")
    print("      for f in *.tif; do")
    print('        gdal_translate -of JPEG -co QUALITY=90 "$f" "${f%.tif}.jpg"')
    print("      done")
    print("=" * 70)

    save_csv()
    save_manifest(all_tasks, exported, skipped, total_pot)
    save_task_ids(all_tasks)

    print(f"\n  ✅  Script complete.  {exported} tasks submitted to GEE.")
    print("      Files will appear in Google Drive as tasks finish.\n")


if __name__ == '__main__':
    main()

  HASDEO FOREST — GeoTIFF DOWNLOADER v2  (TARGET: 1 000+ IMAGES)
  Export mode      : DRIVE
  Resolution       : 10 m
  Max cloud cover  : 20%
  Regions          : 7  → ['Hasdeo_Full', 'PEKB_Core', 'Kente_Extension', 'Control_Forest', 'Hasdeo_North', 'Hasdeo_South', 'Mining_Buffer']
  Monthly periods  : 152 (Jan 2013 – Aug 2025)
  Potential exports: 1,323
    ├─ Monthly composites : 1064
    ├─ Annual mosaics     : 91
    └─ Seasonal (dry+wet) : 168
✓ Earth Engine initialized | project: glacier-probe-model-475519
✓ Output directories ready under: hasdeo_training_dataset/

[ PART 1 ]  Monthly composites — all years 2013-2025
------------------------------------------------------------------------

  🌳 Region: Hasdeo_Full


  Hasdeo_Full: 100%|███████████████████████| 152/152 [03:51<00:00,  1.53s/month]



  🌳 Region: PEKB_Core


  PEKB_Core: 100%|█████████████████████████| 152/152 [03:06<00:00,  1.23s/month]



  🌳 Region: Kente_Extension


...


  🌳 Region: Hasdeo_South


  Hasdeo_South: 100%|██████████████████████| 152/152 [03:04<00:00,  1.21s/month]



  🌳 Region: Mining_Buffer


  Mining_Buffer: 100%|█████████████████████| 152/152 [03:03<00:00,  1.21s/month]




[ PART 2 ]  Annual mosaics (2013–2025)
------------------------------------------------------------------------

  🌳 Region: Hasdeo_Full
    2013: no scenes — skipped
    2014: no scenes — skipped
    2015: no scenes — skipped
    ✓ 2016: 3 scenes → ANN_Hasdeo_Full_2016
    ✓ 2017: 18 scenes → ANN_Hasdeo_Full_2017
    ✓ 2018: 24 scenes → ANN_Hasdeo_Full_2018
    ✓ 2019: 71 scenes → ANN_Hasdeo_Full_2019
    ✓ 2020: 63 scenes → ANN_Hasdeo_Full_2020
    ✓ 2021: 74 scenes → ANN_Hasdeo_Full_2021
    ✓ 2022: 73 scenes → ANN_Hasdeo_Full_2022
    ✓ 2023: 65 scenes → ANN_Hasdeo_Full_2023
    ✓ 2024: 64 scenes → ANN_Hasdeo_Full_2024
    ✓ 2025: 89 scenes → ANN_Hasdeo_Full_2025

  🌳 Region: PEKB_Core
    2013: no scenes — skipped
    2014: no scenes — skipped
    2015: no scenes — skipped
    ✓ 2016: 2 scenes → ANN_PEKB_Core_2016
    ✓ 2017: 9 scenes → ANN_PEKB_Core_2017
    ✓ 2018: 15 scenes → ANN_PEKB_Core_2018
    ✓ 2019: 35 scenes → ANN_PEKB_Core_2019
    ✓ 2020: 31 scenes → ANN_PEKB_Core_2

In [18]:
"""
Hasdeo Forest JPEG Downloader — 1000+ IMAGE DATASET
=====================================================
Exports RGB Sentinel-2 composites to Google Drive for the Hasdeo Arand
forest region. Files sync automatically to your local PC via Google Drive
desktop app (no manual download needed).

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMAGE COUNT BREAKDOWN
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Monthly composites   : 120 months × 7 regions  =  840
  Seasonal dry         :  10 years  × 7 regions  =   70
  Seasonal wet         :  10 years  × 7 regions  =   70
  ─────────────────────────────────────────────────────
  TOTAL POTENTIAL                                = 980+
  (actual count depends on cloud availability per period)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TIMELINE & PHASES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  pre_event    → 2015-01 – 2017-12  (baseline / pre-deforestation)
  first_event  → 2018-01 – 2023-12  (broad clearance window)
  second_event → 2022-03 – 2022-04  (peak clearance, nested tag)
  post_event   → 2024-01 – 2024-12  (recovery monitoring)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
OUTPUT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Drive folder  : Hasdeo_1000_Dataset/
  Format        : RGB GeoTIFF (B4/B3/B2, 30 m) — opens as colour image
  File size     : ~30–60 MB per image at 30 m resolution
  Naming        : MON_<Region>_<YYYY>_<MM>_<phase>.tif
                  SEAS_<DRY|WET>_<Region>_<YYYY>_<phase>.tif
  Local sync    : Install Google Drive desktop app → files appear in
                  your local "Google Drive" folder automatically.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LOCAL DATASET STRUCTURE (after Drive sync)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Google Drive/Hasdeo_1000_Dataset/   ← auto-synced to your PC
  ├── MON_Hasdeo_Full_2015_01_pre_event.tif
  ├── MON_Hasdeo_Full_2022_03_second_event.tif
  ├── SEAS_DRY_Hasdeo_Core_2021_first_event.tif
  └── ... (980+ files)

  Locally generated metadata (run location):
  hasdeo_1000_local/
  ├── metadata/
  │   ├── export_log.csv    ← per-image record (filename, phase, dates, task_id)
  │   └── manifest.json     ← full dataset summary
  └── logs/
      └── task_ids.json     ← all GEE task IDs for monitoring

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
SETUP & USAGE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  1. pip install earthengine-api tqdm
  2. earthengine authenticate        # one-time browser login
  3. python hasdeo_1000_downloader.py
  4. Monitor tasks: https://code.earthengine.google.com/tasks
  5. Files appear in Google Drive → syncs locally via desktop app
"""

import ee
import csv
import json
import time
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID        = 'glacier-probe-model-475519'  # ← your GEE Cloud project ID
OUTPUT_DIR        = 'hasdeo_1000_local'           # local metadata/logs directory
MAX_CLOUD_COVER   = 20                            # % cloud cover filter per scene
MIN_IMAGES_PERIOD = 1                             # skip period if fewer scenes
SCALE             = 30                            # 30 m → ~30–60 MB per image

# ── Single flat Drive folder — ALL images land here ──────────────────────
# Google Drive desktop app syncs this folder to your local PC automatically.
DRIVE_FOLDER = 'Hasdeo_1000_Dataset'

# ── 7 sub-regions covering Hasdeo Arand (WGS84: [west, south, east, north])
REGIONS = {
    'Hasdeo_Full':    [82.58, 22.40, 83.20, 22.90],  # entire block
    'Hasdeo_North':   [82.58, 22.65, 83.20, 22.90],  # northern half
    'Hasdeo_South':   [82.58, 22.40, 83.20, 22.65],  # southern half
    'Hasdeo_Core':    [82.75, 22.50, 83.05, 22.80],  # dense forest core
    'Hasdeo_East':    [82.95, 22.40, 83.20, 22.90],  # eastern edge
    'Hasdeo_West':    [82.58, 22.40, 82.90, 22.90],  # western edge
    'Hasdeo_Buffer':  [82.50, 22.30, 83.30, 23.00],  # extended buffer zone
}

# ── Bands selected for cloud masking (RGB export only needs B2/B3/B4) ────
S2_BANDS = ['B2', 'B3', 'B4', 'SCL']

# ─────────────────────────────────────────────────────────────────────────────
# PHASE LABELLING
# ─────────────────────────────────────────────────────────────────────────────

def get_phase(year: int, month: int) -> str:
    """
    Assign a deforestation phase label to each month composite.
      pre_event    → 2015–2017  (baseline forest cover)
      second_event → Mar–Apr 2022  (peak clearance event — nested within first)
      first_event  → 2018–2023  (broad deforestation window)
      post_event   → 2024       (recovery / monitoring phase)
    """
    if 2015 <= year <= 2017:
        return 'pre_event'
    if year == 2022 and month in (3, 4):
        return 'second_event'
    if 2018 <= year <= 2023:
        return 'first_event'
    if year == 2024:
        return 'post_event'
    return 'background'

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIOD BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    """
    Build monthly windows Jan 2015 – Dec 2024.
    10 years × 12 months = 120 periods per region.
    """
    periods = []
    for year in range(2015, 2025):          # 2015 → 2024 inclusive
        for month in range(1, 13):
            start = f'{year}-{month:02d}-01'
            end   = (f'{year + 1}-01-01'
                     if month == 12
                     else f'{year}-{month + 1:02d}-01')
            periods.append((start, end, get_phase(year, month), year, month))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# EARTH ENGINE INITIALISATION
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialised  |  project: {PROJECT_ID}")
    except Exception:
        print("⚠  Authenticating with Earth Engine (browser will open)...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialised.")

def create_local_dirs():
    """Create local directories for metadata and logs."""
    for sub in ['metadata', 'logs']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Local dirs ready: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    """
    Dual-layer cloud mask:
      QA60  — opaque cloud (bit 10) + cirrus (bit 11)
      SCL   — shadow (3), med-cloud (8), high-cloud (9),
               cirrus (10), snow/ice (11)
    """
    qa       = image.select('QA60')
    qa_mask  = (qa.bitwiseAnd(1 << 10).eq(0)
                  .And(qa.bitwiseAnd(1 << 11).eq(0)))
    scl      = image.select('SCL')
    scl_mask = (scl.neq(3) .And(scl.neq(8))
                            .And(scl.neq(9))
                            .And(scl.neq(10))
                            .And(scl.neq(11)))
    return image.updateMask(qa_mask.And(scl_mask))


def get_s2_collection(roi, start_date, end_date):
    """
    Return a cloud-masked Sentinel-2 SR collection.
    Selects only bands needed (B2/B3/B4 for RGB + SCL + QA60 for masking).
    Keeping the band selection minimal speeds up GEE processing.
    """
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['QA60'])
              .map(mask_s2_clouds))

# ─────────────────────────────────────────────────────────────────────────────
# RGB EXPORT TO DRIVE
# ─────────────────────────────────────────────────────────────────────────────

def export_rgb_to_drive(composite, description, roi):
    """
    Visualise composite as RGB (B4/B3/B2) and export to Drive.

    Why GeoTIFF instead of JPEG?
    GEE's toDrive() only supports GeoTIFF and TFRecord formats — native
    JPEG export is not available via the batch API. The resulting file is
    a 3-band (Uint8) RGB GeoTIFF that opens identically to a JPEG in
    any image viewer, QGIS, Photoshop, or Python (PIL/rasterio).

    At 30 m resolution, expected file size per image: ~30–60 MB.

    To convert to true .jpg after Drive sync (requires GDAL):
        gdal_translate -of JPEG -co QUALITY=90 input.tif output.jpg
    """
    rgb = composite.visualize(
        bands=['B4', 'B3', 'B2'],
        min=200,
        max=3000,
        gamma=1.4,
    )
    task = ee.batch.Export.image.toDrive(
        image          = rgb,
        description    = description,    # task label in GEE console
        folder         = DRIVE_FOLDER,   # flat Drive folder (no nesting)
        fileNamePrefix = description,    # filename = description + .tif
        region         = roi,
        scale          = SCALE,          # 30 m
        crs            = 'EPSG:32644',   # UTM Zone 44N — Hasdeo, Chhattisgarh
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True},
    )
    task.start()
    return task

# ─────────────────────────────────────────────────────────────────────────────
# SEASONAL COMPOSITE BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def make_seasonal_composite(roi, year, season):
    """
    Median composite for a season window:
      dry → Nov–Apr  (low cloud cover; optimal for deforestation detection)
      wet → May–Oct  (captures phenological/greenness changes)
    Returns (composite, scene_count, start_str, end_str).
    """
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'
    col   = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    return col.median().clip(roi), count, start, end

# ─────────────────────────────────────────────────────────────────────────────
# METADATA TRACKING
# ─────────────────────────────────────────────────────────────────────────────

records = []

def log_record(task_id, description, region,
               start, end, count, phase, ctype):
    """Append one export record to the in-memory list."""
    records.append({
        'filename':       f'{description}.tif',
        'drive_folder':   DRIVE_FOLDER,
        'description':    description,
        'region':         region,
        'composite_type': ctype,
        'phase':          phase,
        'start_date':     start,
        'end_date':       end,
        'scene_count':    count,
        'task_id':        task_id,
        'scale_m':        SCALE,
        'bands':          'B4,B3,B2 (RGB visualised, Uint8)',
        'crs':            'EPSG:32644',
        'approx_size_mb': '30–60',
        'exported_at':    datetime.now().isoformat(),
    })


def save_csv():
    if not records:
        return
    p = f'{OUTPUT_DIR}/metadata/export_log.csv'
    with open(p, 'w', newline='') as f:
        w = csv.DictWriter(f, fieldnames=records[0].keys())
        w.writeheader()
        w.writerows(records)
    print(f"✓ CSV     → {p}  ({len(records)} records)")


def save_manifest(all_tasks, exported, skipped, potential):
    """Write a full dataset summary JSON."""
    manifest = {
        'dataset':          'Hasdeo 1000 JPEG Dataset',
        'generated_at':     datetime.now().isoformat(),
        'drive_folder':     DRIVE_FOLDER,
        'local_metadata':   OUTPUT_DIR,
        'regions':          list(REGIONS.keys()),
        'timeline':         '2015-01-01 – 2024-12-31',
        'phases': {
            'pre_event':    '2015–2017  (baseline forest cover)',
            'first_event':  '2018–2023  (broad deforestation window)',
            'second_event': '2022-03 – 2022-04  (peak clearance event)',
            'post_event':   '2024       (recovery monitoring)',
        },
        'composite_types':  ['monthly', 'seasonal_dry', 'seasonal_wet'],
        'scale_m':          SCALE,
        'crs':              'EPSG:32644',
        'bands':            'B4, B3, B2  (RGB GeoTIFF, Uint8)',
        'approx_size_mb':   '30–60 per image',
        'summary': {
            'potential_images': potential,
            'tasks_queued':     exported,
            'skipped':          skipped,
            'estimated_gb':     f'{exported * 45 / 1024:.1f} GB  (at ~45 MB avg)',
        },
        'local_sync_note': (
            'Install Google Drive desktop app. Once GEE tasks complete, '
            f'files appear under Google Drive/{DRIVE_FOLDER}/ and sync '
            'automatically to your PC — no manual download required.'
        ),
        'monitor_url':     'https://code.earthengine.google.com/tasks',
        'tasks': [
            {'id': t.id, 'description': t.config.get('description', '')}
            for t in all_tasks if t
        ],
    }
    p = f'{OUTPUT_DIR}/metadata/manifest.json'
    with open(p, 'w') as f:
        json.dump(manifest, f, indent=2)
    print(f"✓ Manifest → {p}")


def save_task_ids(all_tasks):
    p = f'{OUTPUT_DIR}/logs/task_ids.json'
    with open(p, 'w') as f:
        json.dump(
            [{'id': t.id, 'description': t.config.get('description', '')}
             for t in all_tasks if t],
            f, indent=2,
        )
    print(f"✓ Task IDs → {p}")

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():

    # ── Counts ──────────────────────────────────────────────────────────────
    n_regions    = len(REGIONS)
    monthly_pot  = len(TIME_PERIODS) * n_regions   # 120 × 7 = 840
    seasonal_pot = 10 * n_regions * 2              # 10yr × 7 × 2 = 140
    total_pot    = monthly_pot + seasonal_pot       # 980

    print("=" * 70)
    print("  HASDEO FOREST — 1000 IMAGE JPEG DATASET DOWNLOADER")
    print("=" * 70)
    print(f"  Drive folder    : {DRIVE_FOLDER}/")
    print(f"  Local sync      : Google Drive desktop app → auto-sync to PC")
    print(f"  Resolution      : {SCALE} m  (~30–60 MB per image)")
    print(f"  Bands           : B4 / B3 / B2  (RGB, Uint8)")
    print(f"  Timeline        : Jan 2015 – Dec 2024  (10 years)")
    print(f"  Regions         : {n_regions}  → {list(REGIONS.keys())}")
    print(f"  Potential total : {total_pot} images")
    print(f"    ├─ Monthly    : {monthly_pot}   (120 months × {n_regions} regions)")
    print(f"    └─ Seasonal   : {seasonal_pot}   (10 yr × {n_regions} regions × 2 seasons)")
    print(f"  Est. local size : ~{total_pot * 45 // 1024} GB  (at 45 MB avg per image)")
    print("=" * 70)

    init_ee()
    create_local_dirs()

    all_tasks = []
    exported  = 0
    skipped   = 0

    # ════════════════════════════════════════════════════════════════════════
    # PART 1 — Monthly composites  (Jan 2015 – Dec 2024, 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n[ PART 1 ]  Monthly composites  (Jan 2015 – Dec 2024)")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for start, end, phase, year, month in tqdm(
                TIME_PERIODS,
                desc=f"  {region_name[:20]}",
                unit='month'):
            try:
                col   = get_s2_collection(roi, start, end)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PERIOD:
                    skipped += 1
                    continue

                composite = col.median().clip(roi)

                # e.g. MON_Hasdeo_Full_2022_03_second_event
                desc = f'MON_{region_name}_{year}_{month:02d}_{phase}'[:90]

                task = export_rgb_to_drive(composite, desc, roi)
                all_tasks.append(task)
                log_record(task.id, desc, region_name,
                           start, end, count, phase, 'monthly')
                exported += 1
                time.sleep(0.35)   # gentle GEE rate-limiting

            except Exception as e:
                print(f"\n    ✗ [{region_name} {start}]: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # PART 2 — Seasonal composites  (dry + wet, 2015–2024, 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n\n[ PART 2 ]  Seasonal composites  (dry Nov–Apr + wet May–Oct)")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for year in range(2015, 2025):          # 2015–2024
            for season in ('dry', 'wet'):
                try:
                    composite, count, s_date, e_date = make_seasonal_composite(
                        roi, year, season)

                    if count < MIN_IMAGES_PERIOD:
                        print(f"    ⊙ {season} {year}: no scenes — skipped")
                        skipped += 1
                        continue

                    # Determine phase for seasonal label
                    if year <= 2017:
                        phase = 'pre_event'
                    elif year == 2022 and season == 'dry':
                        phase = 'second_event'   # dry-2022 overlaps peak event
                    elif year <= 2023:
                        phase = 'first_event'
                    else:
                        phase = 'post_event'

                    # e.g. SEAS_DRY_Hasdeo_Core_2022_second_event
                    desc = f'SEAS_{season.upper()}_{region_name}_{year}_{phase}'[:90]

                    task = export_rgb_to_drive(composite, desc, roi)
                    all_tasks.append(task)
                    log_record(task.id, desc, region_name,
                               s_date, e_date, count, phase,
                               f'seasonal_{season}')
                    exported += 1
                    print(f"    ✓ {season.upper()} {year}: "
                          f"{count} scenes → {desc}")
                    time.sleep(0.35)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # SUMMARY
    # ════════════════════════════════════════════════════════════════════════
    print("\n" + "=" * 70)
    print("  EXPORT SUMMARY")
    print("=" * 70)
    print(f"  ✓ GEE tasks queued  : {exported} images")
    print(f"  ⊙ Skipped           : {skipped}  (insufficient cloud-free scenes)")
    print(f"  📦 Est. total size  : ~{exported * 45 // 1024} GB  (at ~45 MB avg)")
    print()
    print(f"  📁 Drive folder     : {DRIVE_FOLDER}/")
    print(f"  🔗 Monitor tasks    : https://code.earthengine.google.com/tasks")
    print()
    print("  HOW TO GET FILES ON YOUR LOCAL PC:")
    print("  ─────────────────────────────────────────────────────────────")
    print("  1. Wait for GEE tasks to complete (check monitor link above)")
    print("     Large exports: 10 min – several hours depending on region")
    print("  2. Open Google Drive in browser — files appear in:")
    print(f"     My Drive / {DRIVE_FOLDER}/")
    print("  3. Google Drive desktop app syncs them to your PC automatically")
    print("     Default local path (Windows): C:/Users/<you>/Google Drive/")
    print("                         (Mac)   : ~/Google Drive/")
    print()
    print("  OPTIONAL — convert RGB GeoTIFF → true JPEG (requires GDAL):")
    print("    Windows (PowerShell):")
    print("      Get-ChildItem *.tif | ForEach-Object {")
    print("        gdal_translate -of JPEG -co QUALITY=90 $_.Name ($_.BaseName+'.jpg')")
    print("      }")
    print("    Mac/Linux:")
    print("      for f in *.tif; do")
    print('        gdal_translate -of JPEG -co QUALITY=90 "$f" "${f%.tif}.jpg"')
    print("      done")
    print("=" * 70)

    save_csv()
    save_manifest(all_tasks, exported, skipped, total_pot)
    save_task_ids(all_tasks)

    print(f"\n  ✅  Script complete.  {exported} tasks submitted to GEE.")
    print("      Files will appear in Google Drive as tasks finish.\n")


if __name__ == '__main__':
    main()

✓ Earth Engine initialized for project: glacier-probe-model-475519
🇦🇺 GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER
Multiple Sub-Regions for Complete Coverage
📍 Regions: 5
   • Blue_Mts_Katoomba: [150.25, -33.75, 150.35, -33.65]
   • Blue_Mts_Wentworth_Falls: [150.35, -33.75, 150.45, -33.65]
   • Warragamba_Dam_North: [150.55, -33.95, 150.65, -33.85]
   • Warragamba_Dam_South: [150.55, -34.05, 150.65, -33.95]
   • Penrith_Urban_Edge: [150.65, -33.8, 150.75, -33.7]
🔥 Event: Black Summer Bushfires (2019-2020)
🌵 Context: Prolonged drought + catastrophic fire + urban expansion
📊 Research: Bushfire, Drought, Regrowth, Deforestation
✓ Created output directories in sydney_blue_mountains_fringe_rgb/

Target: 75 images total (~15 per region)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)


🌳 Processing Region: Blue_Mts_Katoomba
📍 Coordinates: [150.25, -33.75, 150.35, -33.65]
🎯 Target: 15 images
-------------------------------------------------------------------------------